# Modellkomprimierung

Dieses Notebook ist Teil des Projekts [EmbedML](https://hahn-schickard.gitbook.io/embedml) und basiert auf den Inhalten des Kapitels [Modellkomprimierung](https://hahn-schickard.gitbook.io/embedml/3_tinyml/3.3_modellkomprimierung). Es ist auf [Colab]() und im [GitHub-Repository von Hahn-Schickard]() zu finden.

Installation der benötigten Bibliotheken in den angegebenen Versionen.

In [1]:
!pip install -q numpy==1.26.4 scikit-learn==1.5.2 tensorflow-gpu==2.10 tensorflow-model-optimization==0.8.0 > /dev/null 2>&1

Das System kann den angegebenen Pfad nicht finden.


In [2]:
import os
import sys

import numpy as np
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

Es werden Ordner erstellt, falls diese noch nicht vohanden sind, in welchen die in diesem Notebook erstellten Modelle gespeichert werden.

In [3]:
if not os.path.exists(os.path.join('assets')):
    os.makedirs(os.path.join('assets'))
    
if not os.path.exists(os.path.join('assets', 'models')):
    os.makedirs(os.path.join('assets', 'models'))
    
if not os.path.exists(os.path.join('assets', 'models', 'tf')):
    os.makedirs(os.path.join('assets', 'models', 'tf'))
    
if not os.path.exists(os.path.join('assets', 'models', 'tflite')):
    os.makedirs(os.path.join('assets', 'models', 'tflite'))

## Baseline Modell

Ein Basline Modell wird trainiert, welches als Grundlage für das Notebook dient.

Im ersten Schritt, wird der MNIST-Datensatz geladen. Die Pixelwerte des MNIST-Datensatzes werden durch 255 geteilt, um sie auf den Bereich von 0 bis 1 zu normalisieren, da die ursprünglichen Werte zwischen 0 und 255 liegen. Im selben Schritt werden die Daten, in den Datentyp float32 konvertiert und in Trainings- und Testdaten aufgeteilt.

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train / 255.0).astype(np.float32)
X_test = (X_test / 255.0).astype(np.float32)

print(X_train.shape, X_test.shape)

(60000, 28, 28) (10000, 28, 28)


Durch die Verwendung von np.expand_dims mit axis=3 wird eine neue Dimension an der vierten Position (Index 3) der Arrays `X_train` und `X_test` hinzugefügt. Durch das Hinzufügen einer zusätzlichen Dimension wird die Form zu (Anzahl_Bilder, Höhe, Breite, Kanäle), was später für das neuronale Netz erforderlich ist.

In [5]:
X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)

print(X_train.shape, X_test.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


Die Funktion `create_model` definiert ein sequentielles neuronales Netzwerk in TensorFlow, das für die Klassifikation von Bildern mit einer Eingabegröße von 28x28x1 Pixeln konzipiert ist. Das Modell beginnt mit einer 2D-Faltungsschicht (Conv2D), um Merkmale aus den Eingabedaten zu extrahieren, gefolgt von einer Max-Pooling-Schicht (MaxPooling2D), die die räumlichen Dimensionen der Daten reduziert. Diese Schichten folgen erneut. Die resultierenden Merkmalskarten werden durch eine Flatten-Schicht in einen eindimensionalen Vektor umgewandelt. Dieser Vektor wird dann durch vollvernetzte Schichten (Dense) verarbeitet, gefolgt von Dropout-Schichten (Dropout) zur Vermeidung von Überanpassung. Abschließend liefert eine Dense-Ausgabeschicht mit 10 Neuronen und der Softmax-Aktivierungsfunktion die Wahrscheinlichkeiten für jede der 10 Klassen. Das Modell wird mit dem Adam-Optimizer trainiert und verwendet die Verlustfunktion sparse_categorical_crossentropy. Als Metrik zur Bewertung der Modellleistung wird die Genauigkeit (accuracy) herangezogen.

In [6]:
# Funktion zur Definition eines neuronalen Netzes
def create_model():

    model = Sequential([
        Conv2D(16, (3, 3), activation="relu", input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(16, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(32, activation="relu"),
        Dropout(0.2),
        Dense(16, activation="relu"),
        Dropout(0.2),
        Dense(10, activation="softmax")
    ])

    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

Hier wird das zuvor definierte Modell mit den Trainingsdaten `X_train` und den zugehörigen Label `y_train`, beispielhaft für 5 Epochen trainiert, wobei 20% der Daten für die Validierung verwendet werden, um die Leistung des Modells während des Trainings zu überwachen und Überanpassung zu vermeiden.

In [7]:
model = create_model()

model.summary()

model.fit(X_train, y_train, epochs=5, validation_split=0.2, batch_size=128)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 16)        2320      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 32)                1

Das ursprüngliche Modell wird mit den Testdaten evaluiert, um die Testgenauigkeit zu berechnen.

In [8]:
_, baseline_model_accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f'Baseline Test Accuracy: {baseline_model_accuracy:.3f}')

Baseline Test Accuracy: 0.980


Das trainierte TensorFlow Modell wird in die Datei `mnist_model.h5` gespeichert.

In [9]:
model.save(os.path.join('assets', 'models', 'tf', 'mnist_model.h5'))

## Quantisierung

In diesem Abschnitt werden drei Quantisierungstechniken vorgestellt:
- Statische Quantisierung (Post-training quantization, PTQ)
- Dynamische Quantisierung
- Quantisierungs-bewusstes Training (Quantization-Aware Training, QAT)

### Statische Quantisierung (Post-training quantization, PTQ)

Das zuvor trainierte Baseline Modell wird geladen.

In [10]:
model = tf.keras.models.load_model(os.path.join('assets', 'models', 'tf', 'mnist_model.h5'))

Die Funktion `representative_dataset_gen` dient dazu, eine repräsentative Stichprobe des Trainingsdatensatzes für die Kalibrierung eines TensorFlow Lite Modells bereitzustellen. Sie iteriert hier über die ersten 500 Datenpunkte des Trainingsdatensatzes `X_train` und gibt jedes Beispiel als Liste mit einem Element zurück. Dies ist erforderlich, da das Modell nur einen Eingabewert erwartet, und die Kalibrierungsfunktion eine Liste von Eingabewerten benötigt.

In [11]:
def representative_dataset_gen(dataset):
    # Erstellen eines tf.data.Dataset und Batch-Größe von 1
    dataset = tf.data.Dataset.from_tensor_slices(dataset).batch(1)
    # Iteriere über die ersten 500 Datenpunkte des Trainingsdatensatzes
    for input_value in dataset.take(500):
        # Das Modell erwartet nur einen Eingabewert, daher wird jedes Beispiel
        # als Liste mit einem einzigen Element zurückgegeben.
        yield [input_value]

Dieser Codeabschnitt konvertiert das Baseline Modell in ein TensorFlow Lite Modell und wendet die Quantisierung an:
- Zuerst wird ein Converter-Objekt initialisiert, das das Modell lädt
- Dann werden Optimierungen für die Konvertierung festgelegt, einschließlich der Verwendung eines repräsentativen Datensatzes (`representative_dataset_gen`) zur Bestimmung der Quantisierungsparameter. Das Modell wird auf int8-Basis quantisiert, was bedeutet, dass Eingaben und Ausgaben während der Inferenz als 8-Bit Ganzzahlen behandelt werden, was Speicherplatz spart und die Ausführungsgeschwindigkeit auf entsprechender Hardware beschleunigen kann.
- Schließlich wird das konvertierte quantisierte Modell `tflite_model_quant` erzeugt.

In [12]:
# Konverter-Objekt wird aus dem Baseline Modell erstellt
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optimierungen werden auf Standardwerte gesetzt, um die Konvertierung zu optimieren
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Festlegen eines repräsentativen Datensatzes für die statische Quantisierung
converter.representative_dataset = lambda: representative_dataset_gen(X_train)

# Festlegen der unterstützten Operationen auf TensorFlow Lite eingebauten int8-Operationen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Festlegen des Eingabetyps für Inferenzen auf 8-Bit Ganzzahlen (uint8)
converter.inference_input_type = tf.uint8

# Festlegen des Ausgabetyps für Inferenzen auf 8-Bit Ganzzahlen (uint8)
converter.inference_output_type = tf.uint8

# Konvertieren des Modells in ein TensorFlow Lite Modell mit Quantisierung
tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\dk100\AppData\Local\Temp\tmpq899wu7c\assets


INFO:tensorflow:Assets written to: C:\Users\dk100\AppData\Local\Temp\tmpq899wu7c\assets
C:\Users\dk100\AppData\Roaming\Python\Python310\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


Das quantisierte TensorFlow Lite Modell wird in die Datei `mnist_model_quant.tflite` im Binärformat geschrieben.

In [13]:
with open(os.path.join('assets', 'models', 'tflite', 'mnist_model_quant.tflite'), 'wb') as f:
    f.write(tflite_model_quant)

Hier wird das quantisierte TensorFlow Lite Modell aus der Datei `mnist_model_quant.tflite` geladen und ein Interpreter initialisiert, um die Eingabe- und Ausgabedetails des Modells abzurufen und Speicher für die Ausführung des Modells zuzuweisen.

In [14]:
# Laden des TensorFlow Lite Interpreters mit dem quantisierten Modell
interpreter = tf.lite.Interpreter(model_path=os.path.join('assets', 'models', 'tflite', 'mnist_model_quant.tflite'))

# Allokieren von Speicher für den Interpreter, um das Modell auszuführen
interpreter.allocate_tensors()

# Abrufen der Details für die Eingabedaten des Modells
input_details = interpreter.get_input_details()

# Abrufen der Details für die Ausgabedaten des Modells
output_details = interpreter.get_output_details()

`X_test_uint8` wird erstellt, indem die Werte von `X_test` auf den Bereich von 0 bis 255 skaliert und dann als 8-Bit Ganzzahlen (uint8) gespeichert werden.

In [15]:
X_test_uint8 = (X_test * 255).astype(np.uint8)

Hier wird über alle quantisierten Testdaten iteriert: Für jedes Bild wird die Eingabe in das TensorFlow Lite Modell geladen, die Inferenz ausgeführt und dann überprüft, ob die vorhergesagte Klasse mit dem tatsächlichen Label `y_test` übereinstimmt. Wenn dies der Fall ist, wird die Anzahl der korrekten Vorhersagen um 1 erhöht.

In [16]:
# Initialisierung der Anzahl korrekter Vorhersagen
correct_predictions = 0

# Iteration über alle Testdaten
for i in range(len(X_test_uint8)):
    # Vorbereitung der Eingabedaten für den Interpreter durch Hinzufügen einer zusätzlichen Dimension
    input_data = np.expand_dims(X_test_uint8[i], axis=0)
    
    # Setzen der Eingabedaten im Interpreter
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # Ausführen der Inferenz
    interpreter.invoke()
    
    # Abrufen der Ausgabedaten vom Interpreter
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Überprüfen, ob die vorhergesagte Klasse mit dem tatsächlichen Label übereinstimmt
    if np.argmax(output_data) == y_test[i]:
        correct_predictions += 1

Die Genauigkeit des quantisierten Modells wird berechnet, indem die Anzahl der korrekten Vorhersagen durch die Gesamtanzahl der Testdaten geteilt wird.

In [17]:
ptq_accuracy = correct_predictions / len(X_test)
print(f'PTQ Test Accuracy: {ptq_accuracy:.3f}')

PTQ Test Accuracy: 0.979


### Dynamische Quantisierung

Das zuvor trainierte Baseline Modell wird geladen.

In [18]:
model = tf.keras.models.load_model(os.path.join('assets', 'models', 'tf', 'mnist_model.h5'))

Ein TensorFlow Lite Converter wird initialisiert, um das Baseline Modell zu konvertieren, wobei Standardoptimierungen angewendet werden. Das konvertierte Modell `tflite_dynamic_quant_model` nutzt dynamische Quantisierung.

In [19]:
# Initialisierung des TensorFlow Lite Converters mit dem Baseline Modell
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Festlegen von Standardoptimierungen für die Konvertierung
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Konvertieren des Modells unter Verwendung von dynamischer Quantisierung
tflite_dynamic_quant_model = converter.convert() 

INFO:tensorflow:Assets written to: C:\Users\dk100\AppData\Local\Temp\tmp74_h5g06\assets


INFO:tensorflow:Assets written to: C:\Users\dk100\AppData\Local\Temp\tmp74_h5g06\assets


Das TensorFlow Lite Modell mit dynamischer Quantisierung wird in die Datei `mnist_dynamic_quant_model.tflite` im Binärformat geschrieben.

In [20]:
with open(os.path.join('assets', 'models', 'tflite', 'mnist_dynamic_quant_model.tflite'), 'wb') as f:
    f.write(tflite_dynamic_quant_model)

Initialisierung des TensorFlow Lite Interpreter-Objekts, um das dynamisch quantisierte Modell aus der Datei `mnist_dynamic_quant_model.tflite` zu laden. Der Speicher für die Ausführung des Modells wird zugewiesen und die Details für die Eingabe- und Ausgabedaten werden abgerufen, um die Inferenz durchführen zu können.

In [21]:
# Laden des TensorFlow Lite Interpreters mit dem dynamisch quantisierten Modell
interpreter = tf.lite.Interpreter(model_path=os.path.join('assets', 'models', 'tflite', 'mnist_dynamic_quant_model.tflite'))

# Allokieren von Speicher für den Interpreter, um das Modell auszuführen
interpreter.allocate_tensors()

# Abrufen der Details für die Eingabedaten des Modells
input_details = interpreter.get_input_details()

# Abrufen der Details für die Ausgabedaten des Modells
output_details = interpreter.get_output_details()

Hier wird eine Schleife über alle Testdatensätze `X_test` durchgeführt: Für jedes Bild wird die Eingabe vorbereitet, indem sie um eine Dimension erweitert und in den TensorFlow Lite Interpreter gesetzt wird. Anschließend wird die Inferenz durchgeführt, die Ausgabe abgerufen und überprüft, ob die vorhergesagte Klasse mit dem tatsächlichen Label `y_test` übereinstimmt. . Wenn dies der Fall ist, wird die Anzahl der korrekten Vorhersagen um 1 erhöht.

In [22]:
# Initialisierung der Zählvariable für korrekte Vorhersagen
correct_predictions = 0

# Iteration über alle Testdaten
for i in range(len(X_test)):
    # Vorbereitung der Eingabedaten für den Interpreter durch Hinzufügen einer zusätzlichen Dimension und Umwandlung in float32
    input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)
    
    # Setzen der Eingabedaten im Interpreter
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # Ausführen der Inferenz
    interpreter.invoke()
    
    # Abrufen der Ausgabedaten vom Interpreter
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Überprüfen, ob die vorhergesagte Klasse mit dem tatsächlichen Label übereinstimmt und Zählen der korrekten Vorhersagen
    if np.argmax(output_data) == y_test[i]:
        correct_predictions += 1

Die Genauigkeit des dynamisch quantisierten Modells wird berechnet, indem die Anzahl der korrekten Vorhersagen durch die Gesamtanzahl der Testdaten geteilt wird.

In [23]:
dynamic_quant_accuracy = correct_predictions / len(X_test)
print(f'Dynamic Quantization Test Accuracy: {dynamic_quant_accuracy:.3f}')

Dynamic Quantization Test Accuracy: 0.980


### Quantisierungs-bewusstes Training (Quantization-Aware Training, QAT)


In diesem Codeabschnitt wird das Quantization-Aware Training (QAT) durchgeführt, indem ein quantisiertes Modell erstellt wird, das die Auswirkungen der Quantisierung bereits während des Trainings berücksichtigt. Das Modell wird mit einem optimierten Optimierer und Verlustfunktion trainiert, um sicherzustellen, dass es sowohl die Quantisierungsanforderungen als auch die Leistungsmetriken erfüllt.

In [24]:
# Import der Funktion zur Quantisierung des Modells aus TensorFlow Model Optimization (tfmot)
quantize_model = tfmot.quantization.keras.quantize_model

# Erstellung eines quantisierten Modells durch Anwendung der Quantisierung auf das Baseline Modell
q_aware_model = quantize_model(create_model())

# Kompilierung des quantisierten Modells
q_aware_model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training des quantisierten Modells
q_aware_model.fit(X_train, y_train, epochs=5, validation_split=0.2, batch_size=128)

Epoch 1/5
375/375 [==============================] - 6s 14ms/step - loss: 0.9257 - accuracy: 0.6914 - val_loss: 0.1840 - val_accuracy: 0.9524
Epoch 2/5
375/375 [==============================] - 5s 12ms/step - loss: 0.3374 - accuracy: 0.8997 - val_loss: 0.1054 - val_accuracy: 0.9689
Epoch 3/5
375/375 [==============================] - 5s 12ms/step - loss: 0.2507 - accuracy: 0.9264 - val_loss: 0.0972 - val_accuracy: 0.9728
Epoch 4/5
375/375 [==============================] - 5s 12ms/step - loss: 0.2073 - accuracy: 0.9388 - val_loss: 0.0839 - val_accuracy: 0.9762
Epoch 5/5
375/375 [==============================] - 5s 12ms/step - loss: 0.1824 - accuracy: 0.9461 - val_loss: 0.0740 - val_accuracy: 0.9796


Das trainierte Modell wird mit den Testdaten evaluiert, um die Testgenauigkeit zu berechnen.

In [25]:
_, q_aware_model_accuracy = q_aware_model.evaluate(X_test, y_test, verbose=0)

print(f'Quantization Aware Training Model Test Accuracy: {q_aware_model_accuracy:.3f}')

Quantization Aware Training Model Test Accuracy: 0.979


Das trainierte (QAT) Modell wird in die Datei `mnist_qat_model.h5` gespeichert.

In [26]:
q_aware_model.save(os.path.join('assets', 'models', 'tf', 'mnist_qat_model.h5'))


Hier wird das quantisierungsbewusste (QAT) Modell in ein TensorFlow Lite (TFLite) Modell konvertiert, wobei Standardoptimierungen für die Konvertierung aktiviert werden, um die Leistung und Effizienz des Modells zu verbessern.

In [27]:
# Initialisierung des TensorFlow Lite Converters mit dem quantisierungsbewussten Modell
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)

# Festlegen von Standardoptimierungen für die Konvertierung, einschließlich der Optimierung der Quantisierung
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Konvertierung des quantisierungsbewussten Modells in ein TensorFlow Lite Modell
tflite_qat_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\dk100\AppData\Local\Temp\tmp4a2nga2d\assets


INFO:tensorflow:Assets written to: C:\Users\dk100\AppData\Local\Temp\tmp4a2nga2d\assets
C:\Users\dk100\AppData\Roaming\Python\Python310\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


Das konvertierte Modell wird in das TensorFlow Lite Format in die Datei `mnist_qat_model_quant.h5` gespeichert.

In [28]:
# Speichern des QAT quantisierten Modells
with open(os.path.join('assets', 'models', 'tflite', 'mnist_qat_model_quant.tflite'), 'wb') as f:
    f.write(tflite_qat_model)

Initialisierung des TensorFlow Lite Interpreter-Objekts, um das quantisierungsbewusste Modell aus der Datei `mnist_qat_model_quant.tflite` zu laden. Der Speicher für die Ausführung des Modells wird zugewiesen und die Details für die Eingabe- und Ausgabedaten werden abgerufen, um die Inferenz durchführen zu können.

In [29]:
# Laden des TensorFlow Lite Interpreters mit dem dynamisch quantisierten Modell
interpreter = tf.lite.Interpreter(model_path=os.path.join('assets', 'models', 'tflite', 'mnist_qat_model_quant.tflite'))

# Allokieren von Speicher für den Interpreter, um das Modell auszuführen
interpreter.allocate_tensors()

# Abrufen der Details für die Eingabedaten des Modells
input_details = interpreter.get_input_details()

# Abrufen der Details für die Ausgabedaten des Modells
output_details = interpreter.get_output_details()

Hier wird eine Schleife über alle Testdatensätze `X_test` durchgeführt: Für jedes Bild wird die Eingabe vorbereitet, indem sie um eine Dimension erweitert und in den TensorFlow Lite Interpreter gesetzt wird. Anschließend wird die Inferenz durchgeführt, die Ausgabe abgerufen und überprüft, ob die vorhergesagte Klasse mit dem tatsächlichen Label `y_test` übereinstimmt. . Wenn dies der Fall ist, wird die Anzahl der korrekten Vorhersagen um 1 erhöht.

In [30]:
# Initialisierung der Zählvariable für korrekte Vorhersagen
correct_predictions = 0

# Iteration über alle Testdaten
for i in range(len(X_test)):
    # Vorbereitung der Eingabedaten für den Interpreter durch Hinzufügen einer zusätzlichen Dimension und Umwandlung in float32
    input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)
    
    # Setzen der Eingabedaten im Interpreter
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # Ausführen der Inferenz
    interpreter.invoke()
    
    # Abrufen der Ausgabedaten vom Interpreter
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Überprüfen, ob die vorhergesagte Klasse mit dem tatsächlichen Label übereinstimmt und Zählen der korrekten Vorhersagen
    if np.argmax(output_data) == y_test[i]:
        correct_predictions += 1

Die Genauigkeit des quantisierungsbewussten Modells wird berechnet, indem die Anzahl der korrekten Vorhersagen durch die Gesamtanzahl der Testdaten geteilt wird.

In [31]:
q_aware_accuracy_tflite = correct_predictions / len(X_test)
print(f'Quantization Aware Test Accuracy: {q_aware_accuracy_tflite:.3f}')

Quantization Aware Test Accuracy: 0.979


## Pruning

In diesem Abschnitt werden zwei Pruningtechniken vorgestellt:
- Gewichtspruning (engl. weight pruning)
- Strukturiertes Pruning (engl. structured pruning)

### Gewichtspruning (engl. weight pruning)

Das zuvor trainierte Baseline Modell wird geladen.

In [32]:
model = tf.keras.models.load_model(os.path.join('assets', 'models', 'tf', 'mnist_model.h5'))

Beim Gewichtspruning werden individuelle Gewichte innerhalb eines neuronalen Netzwerks entfernt, um die Modellgröße zu reduzieren, ohne die Genauigkeit erheblich zu beeinträchtigen. Der folgende Codeabschnitt zeigt, wie man ein Modell mit Gewichtspruning optimieren kann.

In [33]:
# Anwendung von Pruning-Spezifikationen auf das Modell
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0, final_sparsity=0.5, begin_step=2000, end_step=10000)
}

pruned_model = tfmot.sparsity.keras.prune_low_magnitude(create_model(), **pruning_params)

# Kompilierung des geprunten Modells
pruned_model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

pruned_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d_  (None, 26, 26, 16)       306       
 4 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_max_poo  (None, 13, 13, 16)       1         
 ling2d_4 (PruneLowMagnitude                                     
 )                                                               
                                                                 
 prune_low_magnitude_conv2d_  (None, 11, 11, 16)       4626      
 5 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_max_poo  (None, 5, 5, 16)         1         
 ling2d_5 (PruneLowMagnitude                                     
 )                                                    

Das Modell wird mit Pruning für 5 Epochen trainiert.

In [34]:
# Callback für Pruning-Updates während des Trainings
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]

pruned_model.fit(X_train, y_train, epochs=5, validation_split=0.2, batch_size=128, callbacks=callbacks)

Epoch 1/5
  1/375 [..............................] - ETA: 30:37 - loss: 2.3219 - accuracy: 0.0625WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0064s vs `on_train_batch_end` time: 0.0352s). Check your callbacks.


375/375 [==============================] - 9s 11ms/step - loss: 0.8888 - accuracy: 0.7070 - val_loss: 0.1788 - val_accuracy: 0.9533
Epoch 2/5
375/375 [==============================] - 4s 12ms/step - loss: 0.3607 - accuracy: 0.8910 - val_loss: 0.1195 - val_accuracy: 0.9663
Epoch 3/5
375/375 [==============================] - 5s 12ms/step - loss: 0.2845 - accuracy: 0.9159 - val_loss: 0.1004 - val_accuracy: 0.9703
Epoch 4/5
375/375 [==============================] - 4s 11ms/step - loss: 0.2386 - accuracy: 0.9291 - val_loss: 0.0895 - val_accuracy: 0.9756
Epoch 5/5
375/375 [==============================] - 4s 11ms/step - loss: 0.2028 - accuracy: 0.9406 - val_loss: 0.0825 - val_accuracy: 0.9778


Das geprunte Modell wird mit den Testdaten evaluiert, um die Testgenauigkeit zu berechnen.

In [35]:
_, pruned_model_accuracy = pruned_model.evaluate(X_test, y_test, verbose=0)

print(f'Pruned Test Accuracy: {pruned_model_accuracy:.4f}')

Pruned Test Accuracy: 0.9764


Das trainierte Modell (Gewichtspruning) wird in die Datei `mnist_weight_pruning_model.h5` gespeichert.

In [36]:
pruned_model.save(os.path.join('assets', 'models', 'tf', 'mnist_weight_pruning_model.h5'))

Das geprunte Modell wird in ein TensorFlow Lite Modell konvertiert, um die Modellgröße weiter zu reduzieren und die Effizienz zu verbessern.

In [37]:
# Konvertieren des geprunten Modells in ein TensorFlow Lite Modell
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
tflite_model_pruned = converter.convert()

# Speichern des verkleinerten Modells
with open(os.path.join('assets', 'models', 'tflite', 'mnist_weight_pruning_model.tflite'), 'wb') as f:
    f.write(tflite_model_pruned)

INFO:tensorflow:Assets written to: C:\Users\dk100\AppData\Local\Temp\tmp84kg3_5y\assets


INFO:tensorflow:Assets written to: C:\Users\dk100\AppData\Local\Temp\tmp84kg3_5y\assets


Hier wird das TensorFlow Lite Modell `mnist_pruned_model.tflite` geladen und ein Interpreter initialisiert, um die Eingabe- und Ausgabedetails des Modells abzurufen und Speicher für die Ausführung des Modells zuzuweisen.

In [38]:
# Laden des TensorFlow Lite Interpreters mit dem geprunten Modell
interpreter = tf.lite.Interpreter(model_path=os.path.join('assets', 'models', 'tflite', 'mnist_weight_pruning_model.tflite'))

# Allokieren von Speicher für den Interpreter, um das Modell auszuführen
interpreter.allocate_tensors()

# Abrufen der Details für die Eingabedaten des Modells
input_details = interpreter.get_input_details()

# Abrufen der Details für die Ausgabedaten des Modells
output_details = interpreter.get_output_details()

Hier wird eine Schleife über alle Testdaten `X_test` durchgeführt: Für jedes Bild wird die Eingabe vorbereitet, indem sie um eine Dimension erweitert und in den TensorFlow Lite Interpreter gesetzt wird. Anschließend wird die Inferenz durchgeführt, die Ausgabe abgerufen und überprüft, ob die vorhergesagte Klasse mit dem tatsächlichen Label y_test übereinstimmt. Wenn dies der Fall ist, wird die Anzahl der korrekten Vorhersagen `correct_predictions` erhöht.

In [39]:
# Initialisierung der Zählvariable für korrekte Vorhersagen
correct_predictions = 0

# Iteration über alle Testdaten
for i in range(len(X_test)):
    # Vorbereitung der Eingabedaten für den Interpreter durch Hinzufügen einer
    # zusätzlichen Dimension und Umwandlung in float32
    input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)
    
    # Setzen der Eingabedaten im Interpreter
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # Ausführen der Inferenz
    interpreter.invoke()
    
    # Abrufen der Ausgabedaten vom Interpreter
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Überprüfen, ob die vorhergesagte Klasse mit dem tatsächlichen Label übereinstimmt
    if np.argmax(output_data) == y_test[i]:
        correct_predictions += 1

Die Genauigkeit des geprunten Modells wird berechnet, indem die Anzahl der korrekten Vorhersagen durch die Gesamtanzahl der Testdaten geteilt wird.

In [40]:
pruned_model_accuracy = correct_predictions / len(X_test)
print(f'Pruned Model Test Accuracy: {pruned_model_accuracy:.3f}')

Pruned Model Test Accuracy: 0.976


Pruning ist eine effektive Technik zur Reduzierung der Modellgröße und Verbesserung der Effizienz, insbesondere für Anwendungen auf Edge-Geräten. Durch das Entfernen redundanter Gewichte oder Neuronen kann die Inferenzgeschwindigkeit erhöht werden, ohne die Modellgenauigkeit wesentlich zu beeinträchtigen.

### Strukturiertes Pruning (engl. structured pruning)

In diesem Abschnitt wird strukturiertes Pruning angewendet, um die Größe des Modells zu reduzieren, indem ganze Schichten oder Filter entfernt werden, anstatt nur einzelne Gewichte zu prunen. Dabei verwenden wir ein Framework für strukturiertes Pruning und integrieren es in unseren Workflow mit dem MNIST-Datensatz.

Das zuvor trainierte Baseline Modell wird geladen.

In [41]:
model = tf.keras.models.load_model(os.path.join('assets', 'models', 'tf', 'mnist_model.h5'))

Das [Automatic-Structured-Pruning Repository von GitHub](https://github.com/Hahn-Schickard/Automatic-Structured-Pruning) wird heruntergeladen. Anschließend wird das `src` Verzeichnis zum Python-Pfad hinzugefügt und das `pruning` Modul importiert, zur direkten Nutzung.

In [42]:
!git clone https://github.com/Hahn-Schickard/Automatic-Structured-Pruning
sys.path.append("Automatic-Structured-Pruning/src")

import pruning

Cloning into 'Automatic-Structured-Pruning'...


Hier wird das Framework für strukturiertes Pruning verwendet. In diesem Beispiel wird ein Modell mit 50% Gewichts-Pruning in vollvernetzen Schichten und 70% Filter-Pruning in den Faltungsschichten erstellt.

In [43]:
dense_prune_rate = 50  # 50% Pruning für Dense-Schichten
conv_prune_rate = 70   # 70% Pruning für Filter in Convolutional-Schichten

# Erstelle ein Modell mit strukturiertem Pruning
pruned_model = pruning.factor_pruning(model, dense_prune_rate, conv_prune_rate, 'L2', num_classes=10)

# Kompilieren und Trainieren des geprunten Modells
pruned_model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
pruned_model.fit(X_train, y_train, epochs=5, validation_split=0.2, batch_size=128)

Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________


 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 16)        2320      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 32)                12832     
                                                                 
 dropout (

Nach dem Training des strukturiert geprunten Modells wird die Genauigkeit des Modells auf den Testdaten berechnet.

In [44]:
# Evaluieren des verkleinerten Modells
_, pruned_model_accuracy = pruned_model.evaluate(X_test, y_test, verbose=0)
print(f'Structured Pruning Test Accuracy: {pruned_model_accuracy:.3f}')

Structured Pruning Test Accuracy: 0.961


Das trainierte Modell (strukturiertes Pruning) wird in die Datei `mnist_struct_pruning_model.h5` gespeichert.

In [45]:
pruned_model.save(os.path.join('assets', 'models', 'tf', 'mnist_struct_pruning_model.h5'))

Anschließend wird das Modell in das TensorFlow Lite Format konvertiert, um die Effizienz auf mobilen und eingebetteten Geräten zu verbessern.

In [46]:
# Konvertieren des geprunten Modells in TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
tflite_model_pruned = converter.convert()

# Speichern des geprunten Modells im TensorFlow Lite Format
with open(os.path.join('assets', 'models', 'tflite', 'mnist_struct_pruning_model.tflite'), 'wb') as f:
    f.write(tflite_model_pruned)

INFO:tensorflow:Assets written to: C:\Users\dk100\AppData\Local\Temp\tmp6w35yw5x\assets


INFO:tensorflow:Assets written to: C:\Users\dk100\AppData\Local\Temp\tmp6w35yw5x\assets


Zum Abschluss wird das TensorFlow Lite Modell geladen und getestet, um sicherzustellen, dass das strukturelle Pruning die gewünschte Effizienz erreicht hat, ohne die Leistung des Modells erheblich zu beeinträchtigen.

In [47]:
# Laden des TensorFlow Lite Modells und Initialisieren des Interpreters
interpreter = tf.lite.Interpreter(model_path=os.path.join('assets', 'models', 'tflite', 'mnist_struct_pruning_model.tflite'))
interpreter.allocate_tensors()

# Eingabe- und Ausgabedetails abrufen
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Initialisieren der Anzahl korrekter Vorhersagen
correct_predictions = 0

# Testen des verkleinerten Modells
for i in range(len(X_test)):
    input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    if np.argmax(output_data) == y_test[i]:
        correct_predictions += 1

# Berechnung und Ausgabe der Genauigkeit des TensorFlow Lite Modells
structured_pruned_accuracy = correct_predictions / len(X_test)
print(f'Structured Pruned Model Test Accuracy (TFLite): {structured_pruned_accuracy:.4f}')

Structured Pruned Model Test Accuracy (TFLite): 0.9612
